In [ ]:
"""
- This notebook is used to move files for images
- Do it on all dataset

- Need: meta-data merge (of copper and non-copper)
- Image folder

- Logic:
move copper / non-copper images based on meta-data 
(find the date Y of x% copper images, and use that Y to separate train / test)

# add function:
if we've already separate them once, when we want to recut x%
--> merge copper_train, copper_test into a folde -> redo the cutting again

"""

In [1]:
"""
Testing Block
"""

xxx = glob.glob("/data/put_data/seanyu/ccp/Raw/Non_Copper/*.png")
xxx = [os.path.basename(i) for i in xxx]

arr = []
for i in xxx:
    if i in list(df_non_copper.pid):
        pass
    else:
        arr.append(i)

"""
for i in arr:
    shutil.move(src='/data/put_data/seanyu/ccp/Raw/Copper/' + i, 
                             dst='/data/put_data/seanyu/ccp/Raw/i_am_test/' + i)
"""
#a = pd.DataFrame({'pid' : arr})
#a.to_csv("missed_copper.csv")
len(arr)

NameError: name 'glob' is not defined

In [2]:
df_copper.shape

NameError: name 'df_copper' is not defined

In [3]:
import os
import glob
import numpy as np
import scipy as sp
import pandas as pd
import shutil
import datetime
import time

In [4]:
# meta-data of all images
f_meta_copper = '/data/put_data/seanyu/ccp/Copper_meta_revised_0809.csv'
f_meta_non_copper = '/data/put_data/seanyu/ccp/Non_copper_meta_revised_0809.csv'

# source of all images
copper_img_src = '/data/put_data/seanyu/ccp/Raw/Copper/'
non_copper_img_src = '/data/put_data/seanyu/ccp/Raw/Non_Copper/'

# where to put split images
d_base = '/data/put_data/seanyu/ccp/clean_date_cut/'

# how many percentage to become testing set
thres = 0.5

# to kill old images?
kill_old = False

#----------------------#
# start check folder path
if not os.path.exists(d_base):
    os.mkdir(d_base)
else:
    print('base dir has already exist')

if not os.path.exists(d_base + '/thres' + str(int(thres * 100))):
    os.mkdir(d_base + '/thres' + str(int(thres * 100)))
else:
    print('base dir lv2 has already exist')
    
# define target folders
dst_copper_train = d_base + '/thres' + str(int(thres * 100)) + '/copper_train/'
dst_copper_test = d_base + '/thres' + str(int(thres * 100)) + '/copper_test/'
dst_non_copper_train = d_base + '/thres' + str(int(thres * 100)) + '/non_copper_train/'
dst_non_copper_test = d_base + '/thres' + str(int(thres * 100)) + '/non_copper_test/'

if kill_old:
    shutil.rmtree(dst_copper_train)
    shutil.rmtree(dst_copper_test)
    shutil.rmtree(dst_non_copper_train)
    shutil.rmtree(dst_non_copper_test)
else:
    pass

# generate folder
os.mkdir(dst_copper_train) if not os.path.exists(dst_copper_train) else 'NO'
os.mkdir(dst_copper_test) if not os.path.exists(dst_copper_test) else 'NO'
os.mkdir(dst_non_copper_train) if not os.path.exists(dst_non_copper_train) else 'NO'
os.mkdir(dst_non_copper_test) if not os.path.exists(dst_non_copper_test) else 'NO'

# read meta-data
df_copper = pd.read_csv(f_meta_copper)
df_non_copper = pd.read_csv(f_meta_non_copper)

base dir has already exist


In [5]:
# check duplicate
print(df_copper[df_copper.duplicated('pid')])
print('------')
print(df_non_copper[df_non_copper.duplicated('pid')])

Empty DataFrame
Columns: [pid, position_x, position_y, width, height, area, perimeter, is_copper_defect, datetime, white_area, max_gray, avg_gray, srcid, real_defect]
Index: []
------
Empty DataFrame
Columns: [pid, position_x, position_y, width, height, area, perimeter, is_copper_defect, datetime, white_area, max_gray, avg_gray, srcid, real_defect]
Index: []


In [6]:
df_copper['date'] = [time.strptime(i.split(" ")[0], '%Y/%m/%d') for i in list(df_copper.datetime)]
df_non_copper['date'] = [time.strptime(j.split(" ")[0], '%Y/%m/%d') for j in list(df_non_copper.datetime)]

# sort by date and reset the index
df_copper = df_copper.sort_values('date', ascending=False)
df_non_copper = df_non_copper.sort_values('date', ascending=False)
df_copper = df_copper.reset_index(drop = True)
df_non_copper = df_non_copper.reset_index(drop = True)

#df_copper['date'][0] < time.strptime('2016/03/01', '%Y/%m/%d')
cut_date = df_copper.iloc[int(len(df_copper) * thres)]['date']
df_copper_test = df_copper[df_copper['date'] >= cut_date]
df_copper_train = df_copper[df_copper['date'] < cut_date]

df_non_copper_test = df_non_copper[df_non_copper['date'] >= cut_date]
df_non_copper_train = df_non_copper[df_non_copper['date'] < cut_date]

# check ratio of data set
print('ratio of copper train/test: ' + str(len(df_copper_train)/np.float(len(df_copper)) ) + '/' + str(len(df_copper_test)/np.float(len(df_copper)) ))
print('ratio of non-copper train/test: ' + str(len(df_non_copper_train)/np.float(len(df_non_copper)) ) + '/' + str(len(df_non_copper_test)/np.float(len(df_non_copper)) ))


ratio of copper train/test: 0.4966514297863293/0.5033485702136706
ratio of non-copper train/test: 0.3292087311404256/0.6707912688595744


In [7]:
# don't save meta-data --> force use taking data from dir
def copy_images(dir_src, dir_dst, df_go):
    im_list = list(df_go.pid)
    n_fail = 0
    for i in im_list:
        im_src = dir_src + '/' + i
        im_dst = dir_dst + '/' + i
        try:
            shutil.copyfile(src= im_src, dst= im_dst)
        except IOError:
            n_fail += 1
    print('done, numbers of transfer failed: ' + str(n_fail))

copy_images(dir_src = copper_img_src, dir_dst = dst_copper_train, df_go = df_copper_train)
copy_images(dir_src = copper_img_src, dir_dst = dst_copper_test, df_go = df_copper_test)
copy_images(dir_src = non_copper_img_src, dir_dst = dst_non_copper_train, df_go = df_non_copper_train)
copy_images(dir_src = non_copper_img_src, dir_dst = dst_non_copper_test, df_go = df_non_copper_test)

done, numbers of transfer failed: 0
done, numbers of transfer failed: 0
done, numbers of transfer failed: 0
done, numbers of transfer failed: 10245


In [ ]:
###

In [ ]:
non_copper_img_src
im_non_copper_all = glob.glob(non_copper_img_src + '/*.png')
print(len(im_non_copper_all))
print(df_non_copper.shape)
im_non_copper_all = [os.path.basename(i) for i in im_non_copper_all]

In [ ]:
x = list(set(list(df_non_copper.pid) ) - set(im_non_copper_all))

In [ ]:
ind = 0
for i in x:
    if os.path.exists(copper_img_src + i):
        ind += 1
        
ind

In [ ]:
## 139 images should modify the meta-data --> change the is_copper_defect to 'N' (path is correct, current in the non-copper)
x